In [1]:
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Sampler, DPS, DenoiseOperator, AnisotropicOperator

import torch
import numpy as np
from torch.cuda.amp import autocast as autocast
from torchvision import utils

/home/ryuuyou/.conda/envs/dm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [4]:
data_path = "/home/share/CARE/Isotropic_Liver/train_data/data_label.npz"
data_file = np.load(data_path)
lr_data = data_file['X']

In [5]:
one_batch_data = torch.from_numpy(lr_data[0:25]).cuda()
print(one_batch_data.shape)

torch.Size([25, 1, 128, 128])


# model HR

In [3]:
model_hr = Unet(
    dim = 64,
    channels=1,
    dim_mults = (1, 2, 4, 8)
)
diffusion_hr = DPS(
    model_hr,
    image_size = 128,
    timesteps = 1000,
    sampling_timesteps = 1000,
)
sampler = Sampler(diffusion_hr)
hr_checkpoint_path = "/home/share_ssd/ryuuyou/denoising-diffusion/pretrained_y/model_150k_steps_lr1e-5.pt"
sampler.load(path=hr_checkpoint_path)

loading from: [version]:1.5.4; [step]:150000


# Operator

In [6]:
# operator = InverseProblemOperator(
#     diffusion_model=diffusion_lr,
#     checkpoint_path=lr_checkpoint_path
# )
operator = AnisotropicOperator(img_shape=one_batch_data.shape)
# operator = DenoiseOperator()

# Inference

In [7]:
with autocast():
    res = sampler.dps(measurement=one_batch_data, operator=operator)

sampling loop time step: 100%|██████████| 1000/1000 [03:15<00:00,  5.13it/s]


In [8]:
utils.save_image(res, './out/res.png', nrow=int(np.sqrt(res.shape[0])))

In [9]:
utils.save_image(one_batch_data, './out/input.png', nrow=int(np.sqrt(res.shape[0])))